# Warm up 14 - Vehicle Stock in China

TEP4290, Spring 2024

## Preparation: Packages and data imports

Start by importing the necessary packages:

In [34]:
# Write all the necessary imports here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from dynamic_stock_model import DynamicStockModel

Then import the data into a Dataframe using the pandas read_excel function (check the documentation to find out which arguments to use) and clean it.

In [12]:
#Working directory ?
working_directory = os.getcwd()
print(working_directory)

c:\Users\jujug\Documents\Ecole\4. NTNU\MFA2\TEP4290\TEP4290-juliette\Warm_up_14_chinese_vehicle_fleet


In [30]:
# Extracting data from the source file into a panda Dataframe
file = "Warm_up_14_data.xls"
# This file should be placed in the same directory as this notebook (on the JupyterHub server)

# We read the data and store it in a Dataframe called "data", using the read_excel() panda function
data = pd.read_excel(file, sheet_name="CarStockData",usecols="D:P", header =1,)

# Drop the first two rows
data = data.drop(index=[0,1])

# This is just to replace all the void values in "data" by 0, which makes things easier
data.fillna(0, inplace=True)


C:\Users\jujug\AppData\Local\Temp\ipykernel_4740\1874747389.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.fillna(0, inplace=True)


We create two Dataframes from data:
 - historic_data with values up to 2008
 - future_data with values for 2009 and after


In [31]:
historic_data = data[data.Time < 2009]
future_data = data[data.Time >= 2009]

## Inflow-driven model

Calculate outflows, stock change and stock for historic data, using an inflow-driven model:

In [53]:
# Parameters of the lifetime distribution
n_years = len(historic_data["Time"])
lt_mean = np.full(n_years, mean_lifetime)
lt_std = np.full(n_years, stddev_lifetime)

# Reset the index 
historic_data = historic_data.reset_index(drop=True)

# Maintenant crée ton modèle
inflow_driven_model = DynamicStockModel(
    t = historic_data["Time"].values, # Utilise .values pour passer des tableaux propres
    i = historic_data["Inflow"].values, 
    lt = {'Type': 'Normal', 'Mean': lt_mean, 'StdDev': lt_std}
)
# Create a new attribute in the class dynamic stock model
inflow_driven_model = DynamicStockModel(
    t = historic_data["Time"], 
    i = historic_data["Inflow"], 
    lt = {'Type': 'Normal', 'Mean': lt_mean, 'StdDev': lt_std })

# Compute the entire model
Stock_by_cohort, ExitFlag = inflow_driven_model.compute_s_c_inflow_driven()



In [54]:
# Add the outflow in the historic_data

inflow_driven_model.o_c = None
Outflow_by_cohort, ExitFlag = inflow_driven_model.compute_o_c_from_s_c()
outflow_total = Outflow_by_cohort.sum(axis=1)
historic_data["Outflow"] = outflow_total


In [55]:
# Compute the stock change
historic_data["Stock Change"] = historic_data["Inflow"] - historic_data["Outflow"]

#Compute stock
historic_data["Stock"] = historic_data["Stock Change"].cumsum()

## Stock-driven model

Calculate stock change, outflows and inflows for future data, using a stock-driven model.
You need to run the stock-driven model for all years, combining historic stock data from the inflow-driven model, and future stock data from population and car ownership projections.

In [ ]:
# Create the stock driven model

#pas faire avec future
stock_driven_model = DynamicStockModel(t = future_data["Time"], 
                                       s = future_data["Stock"], 
                                       lt = {'Type': 'Normal', 'Mean': lt_mean, 'StdDev': lt_std })

CheckStr, ExitFlag = TestDSMX.dimension_check()
print(CheckStr)



## Final calculations

Perform the calculations for distance driven, fuel consumption and CO2 emissions

## Plot the results

Plot the results of the model for inflows and outflows, stock change, stock, total distance driven, total fuel consumption, and total CO2 emissions

## Exports the results

Export the results to a new Excel file - you can then in the end before submission copy the stock over to the existing data if you prefer to, but it is not required of you

## Interpretation